## IDEA <br>

- Use OMDB API: http://www.omdbapi.com/#usage
- Do the API request with the search, movies with word 'love', 2 pages
- Use this URL with parameters: http://www.omdbapi.com/?i=tt3896198&apikey=aab3947e&s=love&type=movie&page=2
- Like that get the list of titles of the movies (later create column in df)
- Do another API request title by title to get more info about the movies
- Use this URL with parameters: http://www.omdbapi.com/?i=tt3896198&apikey=aab3947e&t=The+Shawshank+Redemption
- Check how to pass the movie name to the end of URL. Format: xx+xx+xxx+xx (words separated by '+')
- Put info from both requests to the df
- Scrap IMDB or Wikkipedia based on title or IMDB ID to get info from there
- Check if some data matching: Release year, Genre, Director, IMDB Rating
- Could also get more data about the director from Wiki or IMDB

## API <br>

- For the API portion of the project will need to make calls to your chosen API, successfully obtain a response, request data, convert it into a Pandas data frame, and export it as a CSV file.

## WEB SCRAPING <br>

- For the web scraping portion of the project, you will need to scrape the HTML from your chosen page, parse the HTML to extract the necessary information, and either save the results to a text (txt) file if it is text or into a CSV file if it is tabular data.

## 1. API

In [10]:
from pandas.io.json import json_normalize
import pandas as pd
import json
import requests
from bs4 import BeautifulSoup
import time

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
#the function to flatten_json several times

def flatten_json(y):
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(y)
    return out

- Doing the search of the titles that include word 'love' in 2 pages

In [4]:
url= 'http://www.omdbapi.com/?i=tt3896198&apikey=aab3947e&s=love&type=movie&page=2'
response = requests.get(url)
response

results = response.json()
results

<Response [200]>

{'Search': [{'Title': 'From Russia with Love',
   'Year': '1963',
   'imdbID': 'tt0057076',
   'Type': 'movie',
   'Poster': 'https://m.media-amazon.com/images/M/MV5BZjBiNGNlNmItZTk2Zi00YjRlLTk1NzEtNDI2YTNmN2EwNDhlXkEyXkFqcGdeQXVyNDY2MTk1ODk@._V1_SX300.jpg'},
  {'Title': 'From Paris with Love',
   'Year': '2010',
   'imdbID': 'tt1179034',
   'Type': 'movie',
   'Poster': 'https://m.media-amazon.com/images/M/MV5BODAwMDFjNjktMWY2Mi00MmVhLWI0MjYtNzg4OTI0NzA5YzBjXkEyXkFqcGdeQXVyNTIzOTk5ODM@._V1_SX300.jpg'},
  {'Title': 'I Love You Phillip Morris',
   'Year': '2009',
   'imdbID': 'tt1045772',
   'Type': 'movie',
   'Poster': 'https://m.media-amazon.com/images/M/MV5BNDkzMjkwNjc5NV5BMl5BanBnXkFtZTcwNzEyOTk5Mw@@._V1_SX300.jpg'},
  {'Title': 'Love, Simon',
   'Year': '2018',
   'imdbID': 'tt5164432',
   'Type': 'movie',
   'Poster': 'https://m.media-amazon.com/images/M/MV5BNTMyZDdiMzUtZjcxNS00Mjc3LTljY2UtYjI4YmY5NzJlYjc1XkEyXkFqcGdeQXVyMTA5OTkwNTc@._V1_SX300.jpg'},
  {'Title': 'Eat Pray Love',


- creating the list of titles with the word 'love'

In [5]:
title_box = results.get('Search')
title_list = [title.get('Title') for title in title_box]
title_list

['From Russia with Love',
 'From Paris with Love',
 'I Love You Phillip Morris',
 'Love, Simon',
 'Eat Pray Love',
 'To Rome with Love',
 'Stuck in Love.',
 'A Lot Like Love',
 'Love Me If You Dare',
 'Wristcutters: A Love Story']

- converting names into the correct format to later pass them to the URL

In [8]:
titles_formatted = [string.replace(" ", "+") for string in title_list]
titles_formatted

['From+Russia+with+Love',
 'From+Paris+with+Love',
 'I+Love+You+Phillip+Morris',
 'Love,+Simon',
 'Eat+Pray+Love',
 'To+Rome+with+Love',
 'Stuck+in+Love.',
 'A+Lot+Like+Love',
 'Love+Me+If+You+Dare',
 'Wristcutters:+A+Love+Story']

- Getting the info about each movie from the list

In [61]:
url_pattern = 'http://www.omdbapi.com/?i=tt3896198&apikey=aab3947e&t='

In [63]:
appended_dfs=[]

for title in titles_formatted:
    url = url_pattern + title
    response = requests.get(url)
    results = response.json()
    flattened_data = json_normalize(results)
    #appending dataframes to the list to later concatenate them to one dataframe
    appended_dfs.append(flattened_data)
    love_movies = pd.concat(appended_dfs)

love_movies

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,...,Metascore,imdbRating,imdbVotes,imdbID,Type,DVD,BoxOffice,Production,Website,Response
0,From Russia with Love,1963,PG,27 May 1964,115 min,"Action, Adventure, Thriller",Terence Young,"Richard Maibaum (screenplay), Johanna Harwood ...","Sean Connery, Daniela Bianchi, Pedro Armendári...",James Bond willingly falls into an assassinati...,...,85,7.4,"109,544",tt0057076,movie,28 Mar 2006,N/A,United Artists,N/A,True
0,From Paris with Love,2010,R,05 Feb 2010,92 min,"Action, Crime, Thriller",Pierre Morel,"Adi Hasak (screenplay), Luc Besson (story)","John Travolta, Jonathan Rhys Meyers, Kasia Smu...","In Paris, a young employee in the office of th...",...,42,6.5,"108,795",tt1179034,movie,08 Jun 2010,"$23,979,741",Lionsgate Films,N/A,True
0,I Love You Phillip Morris,2009,R,07 Jan 2011,98 min,"Biography, Comedy, Crime, Drama, Romance","Glenn Ficarra, John Requa","John Requa, Glenn Ficarra, Steve McVicker (book)","Jim Carrey, Ewan McGregor, Leslie Mann, Rodrig...",A cop turns con man once he comes out of the c...,...,65,6.6,"88,737",tt1045772,movie,05 Apr 2011,"$2,026,852",Roadside Attractions,N/A,True
0,"Love, Simon",2018,PG-13,16 Mar 2018,110 min,"Comedy, Drama, Romance",Greg Berlanti,"Becky Albertalli (based upon the novel ""Simon ...","Nick Robinson, Jennifer Garner, Josh Duhamel, ...",Simon Spier keeps a huge secret from his famil...,...,72,7.6,"87,922",tt5164432,movie,12 Jun 2018,N/A,20th Century Fox,N/A,True
0,Eat Pray Love,2010,PG-13,13 Aug 2010,133 min,"Drama, Romance",Ryan Murphy,"Ryan Murphy (screenplay), Jennifer Salt (scree...","Julia Roberts, I. Gusti Ayu Puspawati, Hadi Su...",A married woman realizes how unhappy her marri...,...,50,5.8,"81,783",tt0879870,movie,23 Nov 2010,"$80,574,010",Sony Pictures,N/A,True
0,To Rome with Love,2012,R,06 Jul 2012,112 min,"Comedy, Music, Romance",Woody Allen,Woody Allen,"Pierluigi Marchionne, Flavio Parenti, Alison P...",The lives of some visitors and residents of Ro...,...,54,6.3,"81,074",tt1859650,movie,15 Jan 2013,"$16,500,000",Sony Pictures Classics,N/A,True
0,Stuck in Love.,2012,R,02 May 2013,97 min,"Comedy, Drama, Romance",Josh Boone,Josh Boone,"Greg Kinnear, Jennifer Connelly, Lily Collins,...","An acclaimed writer, his ex-wife, and their te...",...,49,7.2,"80,150",tt2205697,movie,08 Oct 2013,"$100,000",Millenium Entertainment,N/A,True
0,A Lot Like Love,2005,PG-13,22 Apr 2005,107 min,"Comedy, Drama, Romance",Nigel Cole,Colin Patrick Lynch,"Amanda Peet, Ashton Kutcher, Taryn Manning, Ai...",Unaquainted Emily and Oliver join the mile hig...,...,48,6.6,"65,452",tt0391304,movie,23 Aug 2005,"$21,835,784",Touchstone Pictures,N/A,True
0,Love Me If You Dare,2003,R,28 May 2004,93 min,"Comedy, Drama, Romance",Yann Samuell,Yann Samuell,"Guillaume Canet, Marion Cotillard, Thibault Ve...","As adults, best friends Julien and Sophie cont...",...,45,7.6,"64,787",tt0364517,movie,19 Oct 2004,"$449,282",Paramount Classics,N/A,True
0,Wristcutters: A Love Story,2006,R,02 Nov 2007,88 min,"Comedy, Drama, Fantasy, Romance",Goran Dukic,"Goran Dukic, Etgar Keret (novel)","Will Arnett, Abraham Benrubi, Leslie Bibb, Mar...",A film set in a strange afterlife way station ...,...,62,7.3,"51,998",tt0477139,movie,18 May 1999,N/A,N/A,N/A,True
